In [1]:
import SimpleITK as sitk
import sys, os

In [2]:
def command_iteration(method):
    print(f"{method.GetOptimizerIteration():3} = {method.GetMetricValue():10.5f}")

input_img = 'PKR_T2.nii'
temp_img = 'mni_icbm152_t2_sym_09c_rss.nii'

fixed = sitk.ReadImage(temp_img, sitk.sitkFloat32)
moving = sitk.ReadImage(input_img, sitk.sitkFloat32)

transformDomainMeshSize = [8] * moving.GetDimension()
tx = sitk.BSplineTransformInitializer(fixed, transformDomainMeshSize)

R = sitk.ImageRegistrationMethod()
R.SetMetricAsCorrelation()

R.SetOptimizerAsLBFGSB(gradientConvergenceTolerance=1e-5,
                       numberOfIterations=50,
                       maximumNumberOfCorrections=5,
                       maximumNumberOfFunctionEvaluations=1000,
                       costFunctionConvergenceFactor=1e+7)
R.SetInitialTransform(tx, True)
R.SetInterpolator(sitk.sitkLinear)

R.AddCommand(sitk.sitkIterationEvent, lambda: command_iteration(R))

outTx = R.Execute(fixed, moving)

print("-------")
print(outTx)
print(f"Optimizer stop condition: {R.GetOptimizerStopConditionDescription()}")
print(f" Iteration: {R.GetOptimizerIteration()}")
print(f" Metric value: {R.GetMetricValue()}")

if ("SITK_NOSHOW" not in os.environ):
    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(fixed)
    resampler.SetInterpolator(sitk.sitkLinear)
    resampler.SetDefaultPixelValue(100)
    resampler.SetTransform(outTx)

    out = resampler.Execute(moving)
    simg1 = sitk.Cast(sitk.RescaleIntensity(fixed), sitk.sitkUInt8)
    simg2 = sitk.Cast(sitk.RescaleIntensity(out), sitk.sitkUInt8)
    cimg = sitk.Compose(simg1, simg2, simg1 // 2. + simg2 // 2.)
    sitk.Show(cimg, "ImageRegistration1 Composition")

Initial Parameters:
  0 =   -0.44874
  0 =   -0.45271
  0 =   -0.46657
  0 =   -0.49807
  0 =   -0.49807
  1 =   -0.44495
  1 =   -0.53776
  1 =   -0.53776
  2 =   -0.56813
  2 =   -0.56813
  3 =   -0.57757
  3 =   -0.57757
  4 =   -0.61013
  4 =   -0.61013
  5 =   -0.61363
  5 =   -0.61363
  6 =   -0.63994
  6 =   -0.63994
  7 =   -0.64144
  7 =   -0.64144
  8 =   -0.66267
  8 =   -0.66267
  9 =   -0.63962
  9 =   -0.67248
  9 =   -0.67248
 10 =   -0.67945
 10 =   -0.67945
 11 =   -0.68857
 11 =   -0.68857
 12 =   -0.69187
 12 =   -0.69187
 13 =   -0.70116
 13 =   -0.70116
 14 =   -0.70635
 14 =   -0.70635
 15 =   -0.71050
 15 =   -0.71050
 16 =   -0.71905
 16 =   -0.71905
 17 =   -0.72297
 17 =   -0.72297
 18 =   -0.72985
 18 =   -0.72985
 19 =   -0.73316
 19 =   -0.73316
 20 =   -0.73766
 20 =   -0.73766
 21 =   -0.73966
 21 =   -0.73966
 22 =   -0.74443
 22 =   -0.74443
 23 =   -0.74746
 23 =   -0.74746
 24 =   -0.75028
 24 =   -0.75028
 25 =   -0.75160
 25 =   -0.75160
 26 =   -0.

In [5]:
moving_resampled = sitk.Resample(moving, fixed, outTx, sitk.sitkLinear, 0.0, moving.GetPixelID())

sitk.WriteImage(moving_resampled, input_img + '_re.nii')
sitk.WriteTransform(outTx, input_img + '_tr.tfm')